In [7]:
import nsepy
from nsepy import get_history, get_index_pe_history
from datetime import date, datetime, timedelta
import pandas as pd
import random
import os
import ntpath
import glob
import time
import talib
import  warnings
from ipywidgets import IntProgress
from IPython.display import display


warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [8]:
data_path =  os.path.join(os.getcwd(),"nse_data\\data\\")
data_index = os.path.join(os.getcwd(),"nse_data\\index_data\\")
dump_path =  os.path.join(os.getcwd(),"nse_data\\dump\\")
meta_data =  os.path.join(os.getcwd(),"nse_data\\meta_data\\")
reports_path = os.path.join(os.getcwd(),"nse_data\\reports\\")
nse_indices = os.path.join(os.getcwd(),"nse_data\\nse_indices\\")
# print(nse_indices)

In [9]:
nifty100_df = pd.read_csv(os.path.join(nse_indices, "ind_nifty100list.csv"))
nifty500_df = pd.read_csv(os.path.join(nse_indices, "ind_nifty500list.csv"))
stocks_list = nifty500_df.Symbol.tolist()
len(stocks_list)

501

In [4]:
stocks_list = list(dict.fromkeys(stocks_list))

add_stocks = ['VEDL', 'YESBANK']
skip_stocks = ['BBTC','IRB','M&MFIN','PVR', 'HATHWAY']

stocks_list = stocks_list + add_stocks

stocks_list = sorted(set(stocks_list)-(set(skip_stocks)))
len(stocks_list)

# stocks_list = random.sample(stocks_list, len(stocks_list))


498

In [5]:
def get_server_last_update_date():
    date_for_update = get_history('TCS', date.today()-timedelta(days=10), date.today()+timedelta(days=1), index = False)
    date_for_update.reset_index(inplace = True)
    return date_for_update.Date.max()

server_date = get_server_last_update_date()
print("Server updated till: ",server_date)

start_date = date(2000, 1, 1)
end_date = date.today()

# start_date = date(2011, 5, 5)
# end_date = date(2011, 5, 9)



Server updated till:  2021-05-21


In [6]:
# Function to get script data file path if present
def get_stock_file_path(script):
    for file in glob.glob(data_path + script + '_*.csv'):
        file = file.replace("\\","/")
        return(file)
    
def get_stock_filename(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)
    
# Function to check if the script data file already present or not: True or False
def stock_data_present(script):
    file = get_stock_file_path(script)
    if file:
        return True
    return False

# Function to get stock dataframe from script name
def stock_df_from_csv(script):
    file = get_stock_file_path(script)
    stock_df = pd.read_csv(file)
    return stock_df

# Function to get stock dataframe from script name with Date as Index
def get_df_date_index(stock):
    df = stock_df_from_csv(stock)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', drop=True, inplace = True)
    df.index.name = None
    return df

In [7]:
# pattern  = request.args.get('pattern', False)
# stocks = {}
# for row in stock_symbol:
#     stocks[row[0]] = {'company': row[1]}
#     stocks[row[0]].update ({'symbol': row[0]})
    
stock_symbols = nifty100_df[[ 'Symbol','Company Name']].values.tolist()   
start_date = date(2020, 1, 1)
end_date = date.today()
patterns_df = pd.read_csv(meta_data + 'patterns.csv')
patterns = patterns_df.values.tolist()
cs_df = []


In [8]:
max_count = len(stock_symbols)
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f)  # display the bar

for stock in stock_symbols:
    f.value += 1
    symbol = stock[0]
    df = get_df_date_index(symbol)
    df = df[start_date:end_date]
    
    for talib_pattern in patterns:  
        pattern_function = getattr(talib, talib_pattern[0])
        pattern_name = talib_pattern[1]

        try:
            results = pattern_function(df['Open'], df['High'], df['Low'], df['Close'])
            last = results.tail(1).values[0]
            
            if last>0   :   cs_df.append({  'Stock': symbol,
                                'Pattern':pattern_name,
                                'Sentiment':'bullish'})
            
            elif last<0 :   cs_df.append({  'Stock': symbol,
                                'Pattern':pattern_name,
                                'Sentiment':'bearish'})
            
        except Exception as e:
                print('failed on filename: ', filename)

y = pd.DataFrame(cs_df)        

IntProgress(value=0)

In [9]:
def highlight_cols(x):
    #copy df to new - original data are not changed
    df = x.copy()
    df[df["Sentiment"]=='bullish'] = 'background-color: green'
    df[df["Sentiment"]=='bearish'] = 'background-color: red'
    return df    

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')


y = y.style.apply(highlight_cols, axis=None)



In [10]:
# Candlesticks Patterns Report
update_date = pd.to_datetime(server_date)
datetime_obj = update_date.to_pydatetime().date() 
candlestick_file = reports_path + 'CandleSticks_Patterns_Report_' + datetime_obj.strftime('%d-%m-%Y') + '.xlsx'
y.to_excel(candlestick_file, index = True)
print(candlestick_file, ' Saved to Disk')

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\reports\CandleSticks_Patterns_Report_21-05-2021.xlsx  Saved to Disk
